In [6]:
from Outline_of_DPN_training import DpnTraining
import numpy as np

DATA_FILE_NAME = "trajectory_dict.pickle"
TRAJECTORY_LENGTH = 30 #Approximately 10 seconds
MIDPOINTS = 2 #splits video data into trajectories of length above, but this determines the amount of overlap across trajectories

EPSILON_PERTURBATIONS = False  #if we want the network to predict how to perturb LS vector.
ITERATIONS = 1000 #kinda like epochs?
BATCH_SIZE = 10   #Might be the exact same thing as episodes, up for interpretation.
EPISODES = 20     #How many trajectories to explore for a given job. Essentually to get a better estimate of the expected reward.
DISCOUNT = 0.99   #how much to discount the reward
ALPHA = 0.001     #learning rate?
INPUT_SIZE = 20

training_object = DpnTraining(INPUT_SIZE = INPUT_SIZE)

In [7]:
episode_array = training_object.train(ITERATIONS)

In [8]:
from Outline_of_DPN_training import curried_valuation
import numpy as np

def curried_valuation(length_of_longest_trajectory):
    '''
    Given the length of the longest trajectory of a set of episodes;

    returns the function that will compute the valuation of an episode array (while padding it)

    Result intended to be used as  map(valuation, episodes_array) to return valuation of each episodes.
    '''
    def valuation(episode):
        '''
        returns the valuation of an episode (with padding)
        input: [(s_0, a_0, r_0), ... ,(s_t, a_t, r_t)]         potentially t<length_of_longest_trajectory
        output: [v_0, v_1, ... v_L]
        '''

        length = len(episode)
        if length != length_of_longest_trajectory:
            #If the episode isn't as long as the longest trajectory, pad it
            episode.extend([(0,0,0) for y in range(length_of_longest_trajectory-length)]) #have to make sure the numbers line up correctly
        out = np.zeros(len(episode))
        x = [i[2] for i in episode] #rewards
        out[-1] = x[-1]
        for i in reversed(range(len(x)-1)): #go backwards
            out[i] = x[i] + DISCOUNT*out[i+1] #this step valuation = reward + gamma*next_step_valuation
            #print(out[i])
        #assert x.ndim >= 1
        return out
    return valuation


In [9]:
"""
longest_trajectory = max(len(episode) for episode in episode_array)
valuation_fun = curried_valuation(longest_trajectory)
cum_values = np.array([valuation_fun(ep) for ep in episode_array])
cum_values
"""
#Notice how they are all completely equivalent.
#This is because after the first function call, all other input arrays are completely equivalent.

'\nlongest_trajectory = max(len(episode) for episode in episode_array)\nvaluation_fun = curried_valuation(longest_trajectory)\ncum_values = np.array([valuation_fun(ep) for ep in episode_array])\ncum_values\n'

In [10]:
first = episode_array[0]
second = episode_array[1]
print(first)
print(second)

[(tensor([39.1451, 24.9129, 28.4255, 40.0107,  6.5238,  0.0000,  1.5132, 51.3752,
         0.0000,  0.0000, 20.6812,  0.0000,  0.3226, 49.4065,  0.0000,  0.7123,
         0.0000, 29.2920, 24.5229, 38.0963]), tensor([110.1985, -23.0497,  49.6430,  56.7955,  55.1665,  47.0463,  23.3406,
          2.7097, 311.7632,   9.0414, -61.4686, -84.3039,  58.0935,   3.6658,
        -45.4539, -65.3160,  63.0694,  31.4545, -38.0830, -34.7997]), array(-390.56165, dtype=float32)), (tensor([4.2861e+01, 2.7481e+01, 3.1512e+01, 4.3729e+01, 8.6225e+00, 0.0000e+00,
        2.7995e+00, 5.6342e+01, 0.0000e+00, 0.0000e+00, 2.3056e+01, 0.0000e+00,
        1.6076e+00, 5.2237e+01, 0.0000e+00, 4.0400e-02, 0.0000e+00, 3.3411e+01,
        2.6632e+01, 4.0610e+01]), tensor([ 172.6556,  -21.8134,   54.8558,   62.0439,   39.4279,   51.5834,
          32.2591,    2.0473,  762.5333,    7.6510,  -68.4311,  -60.0850,
        -118.0048,    3.6355,  -50.4421,  -71.3932,   68.1999,   30.3234,
         -56.2955,  -38.0843]), ar

In [11]:
print("Rewards for trajectory 1:")
rewards1 = [ep[2] for ep in first]
print(rewards1)
print("\nRewards for trajectory2:")
rewards2 = [ep[2] for ep in second]
print(rewards2)

Rewards for trajectory 1:
[array(-390.56165, dtype=float32), array(-816.0908, dtype=float32), array(-2252.0476, dtype=float32), array(-531.1577, dtype=float32), array(-434.3377, dtype=float32), array(-221.19635, dtype=float32), array(-302.4511, dtype=float32), array(-405.8494, dtype=float32), array(-257.16187, dtype=float32), array(-243.05777, dtype=float32), array(-170.83504, dtype=float32), array(-210.30586, dtype=float32), array(-318.42847, dtype=float32), array(-189.00174, dtype=float32), array(-176.91359, dtype=float32), array(-190.07797, dtype=float32), array(-179.85744, dtype=float32), array(-168.79794, dtype=float32), array(-168.66183, dtype=float32), array(-164.35979, dtype=float32), array(-165.34288, dtype=float32), array(-152.48843, dtype=float32), array(-143.69398, dtype=float32), array(-129.3214, dtype=float32), array(-132.65741, dtype=float32), array(-136.49362, dtype=float32), array(-139.16785, dtype=float32), array(-146.5646, dtype=float32), array(-139.52644, dtype=floa

In [5]:
#print(valuation_fun(first))
#print(valuation_fun(second))
for episode in episode_array:
    def alt(episode):
        #print(episode)
        length = len(episode)
        out = np.zeros(length)
        x = [i[2] for i in episode] #rewards
        print(x)
        out[-1] = x[-1]
        for i in reversed(range(len(x)-1)): #go backwards
            out[i] = x[i] + DISCOUNT*out[i+1]
        return out
    print(alt(episode)[0])
#alt = np.vectorize(alt)

#x = alt(episode_array)

[array(-44.255024, dtype=float32), array(-31.177961, dtype=float32), array(-26.912933, dtype=float32), array(-30.700754, dtype=float32), array(-32.06538, dtype=float32), array(-29.293821, dtype=float32), array(-34.72585, dtype=float32), array(-37.416286, dtype=float32), array(-34.21243, dtype=float32), array(-32.33177, dtype=float32), array(-31.478401, dtype=float32), array(-34.16411, dtype=float32), array(-33.27262, dtype=float32), array(-40.306686, dtype=float32), array(-34.180355, dtype=float32), array(-30.287786, dtype=float32), array(-36.410114, dtype=float32), array(-37.562027, dtype=float32), array(-31.400923, dtype=float32), array(-24.805967, dtype=float32), array(-47.359844, dtype=float32), array(-29.300404, dtype=float32), array(-41.35979, dtype=float32), array(-34.074284, dtype=float32), array(-29.791824, dtype=float32), array(-35.879482, dtype=float32), array(-28.200096, dtype=float32), array(-32.41346, dtype=float32), array(-35.075615, dtype=float32), array(-37.325592, dty

In [26]:
#As you can see, for loops do not fix it.
print(rewards1)
print(rewards2)

[array(-25.162148, dtype=float32), array(-24.808958, dtype=float32), array(-26.091478, dtype=float32), array(-24.279848, dtype=float32), array(-25.500378, dtype=float32), array(-26.107761, dtype=float32), array(-26.219122, dtype=float32), array(-25.370758, dtype=float32), array(-26.662226, dtype=float32), array(-27.169823, dtype=float32), array(-25.985874, dtype=float32), array(-22.771317, dtype=float32), array(-24.913486, dtype=float32), array(-25.427795, dtype=float32), array(-26.728174, dtype=float32), array(-27.392109, dtype=float32), array(-26.363422, dtype=float32), array(-24.165966, dtype=float32), array(-23.258526, dtype=float32), array(-24.549198, dtype=float32), array(-23.845589, dtype=float32), array(-25.360668, dtype=float32), array(-25.68159, dtype=float32), array(-24.706469, dtype=float32), array(-26.474186, dtype=float32), array(-24.682709, dtype=float32), array(-26.536226, dtype=float32), array(-25.27494, dtype=float32), array(-28.055384, dtype=float32), array(-28.46071

In [96]:
x = i(norm(mu,sigma), 1)
action = x.sample()
action

tensor([   377.8866,    283.9444,    144.1590,    179.7208,  41394.2266,
           202.8864,    320.2062,    385.6936,    655.3764, -17537.1816,
          8776.9473,    200.9408,    146.5633,    397.3027,  -1290.3750,
           197.0949,  -8075.4087,    173.6999,    109.0294,    326.9413])

In [97]:
loss = x.log_prob(action)
loss.backward()

C:\Users\Exiz\anaconda3\lib\site-packages\torch\autograd\__init__.py:132: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  allow_unreachable=True)  # allow_unreachable flag


In [98]:
loss

tensor(-105.9893, grad_fn=<SumBackward1>)

In [103]:
other_action = x.sample()
other_action
hmm = other_action.detach() 
x.log_prob(hmm)

tensor(-99.3508, grad_fn=<SumBackward1>)

In [9]:
x = np.array([0,1,2,3,4])
sum(np.square(x-1))

15